# Web Crawler for Ubisoft Games and Stocks

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re

In [3]:
base_url = 'http://www.vgchartz.com/gamedb/games'

In [51]:
params = {'publisher':'Ubisoft',
          'showcriticscore':1,
          'showtotalsales':1,
          'showdeveloper':1,
          'showcriticscore':1,
          'showreleasedate':1,
          'showlastupdate':0,
          'results':200,
          'order':'ReleaseDate',
          'page':1}

In [5]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.26 Safari/537.36 Core/1.63.6776.400 QQBrowser/10.3.2577.400'}

In [7]:
response = requests.get(base_url, params=params, headers=headers)

In [8]:
soup = BeautifulSoup(response.text, 'html5lib')

In [9]:
game_list = soup.find('div', id='generalBody').tbody.find_all('tr')[3:]

In [61]:
def get_game_info(idx):
    game_info = []
    for i,j in enumerate(game_list[idx].find_all('td')):
        if i not in [0,1,3]:
            game_info.append(j.text.strip())
        if i == 3:
            game_info.append(j.img['alt'])
    return game_info

In [11]:
from tqdm import tqdm_notebook

In [12]:
import time

In [13]:
game_matrix = []

for page in range(9):
    
    params['page'] = page + 1
    response = requests.get(base_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html5lib') 
    game_list = soup.find('div', id='generalBody').tbody.find_all('tr')[3:]
    
    for j in tqdm_notebook(range(len(game_list))):
        game_info = get_game_info(j)
        game_matrix.append(game_info)
        
    time.sleep(30)

## Problems：

- html解析器：试过lxml（标签混乱）, html.parser（标签提前闭合）
    - html5lib，容错率最高，准确解析
- http 429：访问速度过快
    - 设置time.sleep()
    - 5s, 10s 还是太短
    - 60s

In [15]:
game_df = pd.DataFrame(game_matrix)

In [17]:
game_df.columns = ['Name', 'Developer', 'CriticScore', 'UserScore', 'GlobalSale', 'ReleaseDate']

In [40]:
valid_game_df = game_df[game_df['CriticScore'] != 'N/A'][game_df['GlobalSale'] != 'N/A']

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [46]:
valid_game_df['GlobalSale'] = valid_game_df['GlobalSale'].apply(lambda x: x[:-1])

In [47]:
valid_game_df

,Name,Developer,CriticScore,GlobalSale,ReleaseDate
39,The Crew 2,Ivory Tower,6.5,0.53,29th Jun 18
43,Far Cry 5,Ubisoft Montreal,7.2,1.59,27th Mar 18
44,Far Cry 5,Ubisoft,7.9,3.63,27th Mar 18
45,Far Cry 5,Ubisoft,6.3,0.07,27th Feb 18
46,Assassin's Creed Rogue Remastered Read the ...,Ubisoft,6.0,0.03,20th Mar 18
53,Monopoly for Nintendo Switch,Engine Software,5.3,0.23,31st Oct 17
69,Rayman Legends,Ubisoft Montpellier,4.5,0.56,12th Sep 17
70,Mario + Rabbids: Kingdom Battle Read the re...,Ubisoft Milan,7.7,2.18,29th Aug 17
73,Tom Clancy's Ghost Recon Wildlands,Ubisoft Paris,7.0,3.04,07th Mar 17
74,Tom Clancy's Ghost Recon Wildlands,Ubisoft Paris,7.4,0.04,07th Mar 17


In [48]:
game_rates = pd.read_csv('UBI_game_rates.csv')

In [50]:
game_rates[game_rates['Score'] != 'tbd']

,Title,Date,Score
28,Starlink: Battle for Atlas (PS4),"Oct 16, 2018",5.2
29,Starlink: Battle for Atlas (XONE),"Oct 16, 2018",6.3
30,Starlink: Battle for Atlas (Switch),"Oct 16, 2018",7.9
31,For Honor: Marching Fire Edition (PS4),"Oct 16, 2018",4.4
33,For Honor: Marching Fire (PS4),"Oct 16, 2018",6.3
35,Child of Light (Switch),"Oct 11, 2018",7.9
36,Assassin's Creed Odyssey (PS4),"Oct 2, 2018",5.6
37,Assassin's Creed Odyssey (XONE),"Oct 2, 2018",4.9
38,Assassin's Creed Odyssey (PC),"Oct 2, 2018",4.9
42,South Park: The Stick of Truth (Switch),"Sep 25, 2018",8.1
